In [1]:
import numpy as np
import pandas as pd        # For loading and processing the dataset
from sklearn.model_selection import train_test_split
import torch
import math
import matplotlib.pyplot as plt
import matplotlib.colors
import torch.nn.functional as F

In [2]:
# Read the CSV input file and show first 5 rows
df_train = pd.read_csv('data/titanic/train.csv')
df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# We can't do anything with the Name, Ticket number, and Cabin, so we drop them.
df_train = df_train.drop(['PassengerId','Name','Ticket', 'Cabin'], axis=1)

In [4]:
# To make 'Sex' numeric, we replace 'female' by 0 and 'male' by 1
df_train['Sex'] = df_train['Sex'].map({'female':0, 'male':1}).astype(int)

In [5]:
# We replace 'Embarked' by three dummy variables 'Embarked_S', 'Embarked_C', and 'Embarked Q',
# which are 1 if the person embarked there, and 0 otherwise.
df_train = pd.concat([df_train, pd.get_dummies(df_train['Embarked'], prefix='Embarked')], axis=1)
df_train = df_train.drop('Embarked', axis=1)

In [6]:
# We normalize the age and the fare by subtracting their mean and dividing by the standard deviation
age_mean = df_train['Age'].mean()
age_std = df_train['Age'].std()
df_train['Age'] = (df_train['Age'] - age_mean) / age_std

fare_mean = df_train['Fare'].mean()
fare_std = df_train['Fare'].std()
df_train['Fare'] = (df_train['Fare'] - fare_mean) / fare_std

In [7]:
# In many cases, the 'Age' is missing - which can cause problems. Let's look how bad it is:
print("Number of missing 'Age' values: {:d}".format(df_train['Age'].isnull().sum()))

# A simple method to handle these missing values is to replace them by the mean age.
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())

Number of missing 'Age' values: 177


In [8]:
# With that, we're almost ready for training
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,-0.530005,1,0,-0.502163,0,0,1
1,1,1,0,0.571430,1,0,0.786404,1,0,0
2,1,3,0,-0.254646,0,0,-0.488580,0,0,1
3,1,1,0,0.364911,1,0,0.420494,0,0,1
4,0,3,1,0.364911,0,0,-0.486064,0,0,1


In [9]:
# Finally, we convert the Pandas dataframe to a NumPy array, and split it into a training and test set
X_train = df_train.drop('Survived', axis=1).values
y_train = df_train['Survived'].values

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [10]:
X_train, y_train, X_test, y_test = map(torch.tensor, (X_train, y_train, X_test, y_test))

In [11]:
print(X_train.shape, y_train.shape)

torch.Size([712, 9]) torch.Size([712])


In [12]:
def accuracy(y_hat, y):
  pred = torch.argmax(y_hat, dim=1)
  return (pred == y).float().mean()

In [13]:
X_train = X_train.float()
y_train = y_train.long()


In [14]:
X_test = X_test.float()
y_test = y_test.long()

In [15]:
import torch.nn as nn
from torch import optim

In [16]:
class FirstNetwork_v3(nn.Module):
  
  def __init__(self):
    super().__init__()
    torch.manual_seed(0)
    self.net = nn.Sequential(
        nn.Linear(9, 128), 
        nn.ReLU(),
        nn.Linear(128, 256),
        nn.ReLU(),
        nn.Linear(256, 2), 
        nn.Softmax()
    )

  def forward(self, X):
    return self.net(X)

  def predict(self, X):
    Y_pred = self.forward(X)
    return Y_pred

In [17]:
def fit_v2(x, y, model, opt, loss_fn, epochs = 1000):
  
  for epoch in range(epochs):
    loss = loss_fn(model(x), y)
    loss.backward()
    opt.step()
    opt.zero_grad()
    
  return loss.item()

In [18]:
# device = torch.device("cuda")
device = torch.device("cpu")

X_train=X_train.to(device)
y_train=y_train.to(device)
X_test=X_test.to(device)
y_test=y_test.to(device)
fn = FirstNetwork_v3()
fn.to(device)
loss_fn = F.cross_entropy
opt = optim.SGD(fn.parameters(), lr=0.5)

print('Final loss', fit_v2(X_train, y_train, fn, opt, loss_fn))

/Users/deddy/Documents/GitHub/regressGAN/venv/lib/python3.9/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Final loss 0.43821704387664795


In [19]:
Y_pred_train = fn.predict(X_train)
#Y_pred_train = np.argmax(Y_pred_train,1)
Y_pred_val = fn.predict(X_test)
#Y_pred_val = np.argmax(Y_pred_val,1)

accuracy_train = accuracy(Y_pred_train, y_train)
accuracy_val = accuracy(Y_pred_val, y_test)

In [20]:
print("Training accuracy", (accuracy_train))
print("Validation accuracy",(accuracy_val))

Training accuracy tensor(0.8694)
Validation accuracy tensor(0.7821)


In [21]:
df_test = pd.read_csv('data/titanic/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [22]:
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df_test['Sex'] = df_test['Sex'].map({'female':0, 'male':1}).astype(int)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')], axis=1)
df_test = df_test.drop('Embarked', axis=1)
df_test['Age'] = (df_test['Age'] - age_mean) / age_std
df_test['Fare'] = (df_test['Fare'] - fare_mean) / fare_std
df_test.head()
X_test = df_test.drop('PassengerId', axis=1).values

In [23]:
X_test = torch.tensor(X_test)

In [24]:
X_test = X_test.float()

In [25]:
X_test=X_test.to(device)

In [26]:
X_test.shape

torch.Size([418, 9])

In [27]:
Y_pred_val = fn.predict(X_test)

/Users/deddy/Documents/GitHub/regressGAN/venv/lib/python3.9/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [28]:
pred = torch.argmax(Y_pred_val, dim=1)

In [29]:
pred.shape

torch.Size([418])

In [30]:
df_test['Survived'] = pred.to('cpu')

In [31]:
output = pd.DataFrame()
output['PassengerId'] = df_test['PassengerId']
output['Survived'] = df_test['Survived'].astype(int)
# output.to_csv('./prediction.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
